In [4]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CAFI'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('batimento')
logger.setLevel(logging.DEBUG)

# fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_batimento.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [16]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('batimento/lev_fisico/Inventário Almoxarifado - Afogados da Ingazeira - 2023 (respostas).xlsx', sheet_name=1)
levantamento = levantamento.rename(columns={"Código":"codigo"})
display(levantamento.info())
levantamento = levantamento[["codigo", "ED", "Nome", "Quantidade", "CADMAT", "valor unitário"]]
levantamento = levantamento.drop_duplicates(keep="last")
display(levantamento.info())

suap = pd.read_excel('batimento/suap/2024-01-08_15-39_CAFI_relatorio_almox.xlsx')
display(suap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  475 non-null    datetime64[ns]
 1   Endereço de e-mail                    475 non-null    object        
 2   Retificação                           0 non-null      float64       
 3   ED                                    475 non-null    int64         
 4   CÓD.FINAL.MAT                         475 non-null    int64         
 5   Observações                           475 non-null    object        
 6   ordenaçãolista                        0 non-null      float64       
 7   codigo                                475 non-null    object        
 8   Unnamed: 8                            475 non-null    bool          
 9   Nome                                  475 non-null    object        
 10  Un

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 0 to 474
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   codigo          473 non-null    object 
 1   ED              473 non-null    int64  
 2   Nome            473 non-null    object 
 3   Quantidade      473 non-null    int64  
 4   CADMAT          412 non-null    float64
 5   valor unitário  0 non-null      float64
dtypes: float64(2), int64(2), object(2)
memory usage: 25.9+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   codigo     411 non-null    object 
 1   descricao  411 non-null    object 
 2   und        411 non-null    object 
 3   qtd        411 non-null    int64  
 4   vlr_total  411 non-null    float64
 5   vlr_unt    411 non-null    float64
 6   ed         411 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 22.6+ KB


None